Questão 2 - Método de Runge-Kutta 4, aplicado em equações de estado.

### Bibliotecas importadas


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Equações de Estado do CKT

A seguinte celula do programa define as equações de estado que regem o circuito do problema.


In [ ]:
def Eq_Estados(t, X, R, L, C, E_i):
    
    i_L = X[0] 
    v_C = X[1] 
    
    
    di_dt = (-R/L) * i_L - (1/L) * v_C + (1/L) * E_i
    dv_dt = (1/C) * i_L
    
    return np.array([di_dt, dv_dt])

### Algoritmo para Runge-Kutta de 4ª Ordem 

O Método de Runge-Kutta de quarta ordem é obtido considerando a série de Taylor para y, desenvolvida até o termo de ordem quatro. O algoritmo calcula quatro estimativas de inclinação ($k_1, k_2, k_3, k_4$) em diferentes pontos dentro do passo de integração $h$. O próximo valor da variável de estado, $y_{i+1}$, é determinado por uma média ponderada dessas inclinações, conforme a equação recursiva:

$$y_{i+1} = y_i + \frac{h}{6}(k_1 + 2k_2 + 2k_3 + k_4)$$


